<a href="https://colab.research.google.com/github/SugarC21/colab_Open_WebUI/blob/main/Colab_Open_WebUI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Open WebUI + Ollama in Colab**

This notebook sets up **Open WebUI** (Python 3.11) and **Ollama** on Google Colab with optional Google Drive persistence.

## **Key Adjustments**
1. **Google Drive is mounted with `noexec`**. Executing binaries directly from Drive triggers a **`PermissionError`**.
2. **We store** (persist) the **Ollama** binary in Drive **for reuse**, but **copy** it to **local disk** (`/content`) at runtime to actually execute it.
3. **Open WebUI** environment can still live in Drive if desired. We reference `open-webui` using an **absolute path** so it won't cause `FileNotFoundError`.
4. **ngrok** is used to expose only the Open WebUI interface (port **8081**). Ollama remains on port **11422**, hidden.


In [ ]:
#@title **Setup**
use_gdrive = True #@param {type:"boolean"}
use_ngrok_auth = True #@param {type:"boolean"}

import os

BASE_PATH = "/content"
if use_gdrive:
    from google.colab import drive
    drive.mount('/content/drive')
    BASE_PATH = "/content/drive/MyDrive/Open-WebUI"
    os.makedirs(BASE_PATH, exist_ok=True)

print("Using base path:", BASE_PATH)
print("Using ngrok auth token:", use_ngrok_auth)

## (Optional) ngrok Authentication Token
If you want a stable subdomain or advanced ngrok features:
1. Toggle `use_ngrok_auth` above.
2. Provide `ngrok_auth_token` via **Colab secrets** or an **environment variable**.


## Install Dependencies
- Installs **Python 3.11**, **venv**, **dev** packages.
- Installs **pciutils** and **lshw**.
- The system is ephemeral, so if you do **not** use Google Drive, everything resets on runtime end.

In [ ]:
!sudo apt-get update -y
!sudo apt-get install -y python3.11 python3.11-venv python3.11-dev pciutils lshw

## Install or Retrieve Ollama
**Key Point**: Because Drive is mounted with `noexec`, we can't run binaries directly from Drive.
1. If `use_gdrive` is **True**, we'll store the Ollama binary in Drive (for persistence)
2. On each run, we **copy** it from Drive to `/content/ollama.bin`, then execute it from local ephemeral disk.
3. If not using Drive, we install Ollama system-wide each time and run it from `/usr/local/bin/ollama` (or similar).

In [ ]:
import os
import subprocess

ollama_drive_dir = os.path.join(BASE_PATH, 'ollama')
ollama_drive_bin = os.path.join(ollama_drive_dir, 'ollama')
ollama_local_bin = "/content/ollama.bin"  # We'll copy from Drive or system-wide here

if use_gdrive:
    if os.path.exists(ollama_drive_bin):
        print("Ollama binary found in Drive.")
        print("Copying it to local disk to bypass 'noexec'...")
        !cp "$ollama_drive_bin" "$ollama_local_bin"
        !chmod +x "$ollama_local_bin"
    else:
        print("No Ollama binary in Drive. Installing Ollama system-wide...")
        !curl -fsSL https://ollama.com/install.sh | sh
        which_ollama = subprocess.check_output(['which', 'ollama']).decode().strip()
        print("Copying system-wide ollama to Drive AND local disk...")
        os.makedirs(ollama_drive_dir, exist_ok=True)
        !cp "$which_ollama" "$ollama_drive_bin"
        !chmod +x "$ollama_drive_bin"
        !cp "$which_ollama" "$ollama_local_bin"
        !chmod +x "$ollama_local_bin"
        print("Ollama persisted in Drive and copied to local disk.")
else:
    print("Installing Ollama system-wide (ephemeral)")
    !curl -fsSL https://ollama.com/install.sh | sh
    print("Copying ephemeral install to local disk...")
    which_ollama = subprocess.check_output(['which', 'ollama']).decode().strip()
    !cp "$which_ollama" "$ollama_local_bin"
    !chmod +x "$ollama_local_bin"
    print("Ollama is ready at", ollama_local_bin)

## Set Up Virtual Environment & Install Open WebUI
We can store **Open WebUI** in Drive for persistence. Running Python from Drive is allowed, but we must reference it via an absolute path.

In [ ]:
venv_path = os.path.join(BASE_PATH, 'venv')
if not os.path.exists(venv_path):
    print("Creating Python 3.11 virtual environment...")
    !python3.11 -m venv "$venv_path"

print("Upgrading pip in venv...")
!"{venv_path}/bin/python" -m pip install --upgrade pip

print("Installing Open WebUI...")
!"{venv_path}/bin/pip" install open-webui

print("Open WebUI installation complete.")

## Create a Script to Start Both Servers
- **Ollama** will be run from the local ephemeral binary at `/content/ollama.bin`.
- **Open WebUI** will be called via the **absolute path** to the `open-webui` CLI in the virtual environment.
- We use Python’s `subprocess` to launch each in separate threads.

In [ ]:
import os

server_script_path = os.path.join(BASE_PATH, 'start_servers.py')
ollama_local_bin = "/content/ollama.bin"  # from above steps
open_webui_bin = os.path.join(venv_path, 'bin', 'open-webui')

# We'll run everything from any directory, so use absolute paths.
script_content = f'''\
import subprocess, threading, time

OLLAMA_CMD = "{ollama_local_bin}"
OPEN_WEBUI_CMD = "{open_webui_bin}"

def start_ollama():
    subprocess.run([OLLAMA_CMD, 'serve'])

def start_open_webui():
    subprocess.run([OPEN_WEBUI_CMD, 'serve', '--port', '8081'])

threading.Thread(target=start_ollama).start()
time.sleep(5)
threading.Thread(target=start_open_webui).start()
'''

with open(server_script_path, 'w') as f:
    f.write(script_content)

print("Created:", server_script_path)
print("- Ollama will run from /content/ollama.bin")
print(f"- Open WebUI will run from {open_webui_bin}")

## Start Servers & Expose Open WebUI via ngrok
The Open WebUI is on **port 8081**, Ollama on **port 11422** (hidden).

In [ ]:
!pip install pyngrok --quiet

import time, os
from pyngrok import ngrok
from google.colab import userdata

def get_ngrok_token():
    secret_token = userdata.get('ngrok_auth_token')
    if secret_token:
        return secret_token
    env_token = os.environ.get('ngrok_auth_token', '')
    if env_token:
        return env_token
    return ''

if use_ngrok_auth:
    token = get_ngrok_token()
    if token:
        ngrok.set_auth_token(token)
        print("ngrok auth token set.")
    else:
        print("No 'ngrok_auth_token' found. Proceeding without auth token.")

server_script_path = os.path.join(BASE_PATH, 'start_servers.py')
venv_path = os.path.join(BASE_PATH, 'venv')
python_bin = os.path.join(venv_path, 'bin', 'python')

# Spawn the script in background using Python's subprocess
p = subprocess.Popen([python_bin, server_script_path])

# Wait briefly for servers to initialize
time.sleep(20)

print("\nAttempting to open ngrok tunnel for Open WebUI (port 8081)...")
webui_tunnel = ngrok.connect(8081, "http")
print("Open WebUI URL:", webui_tunnel.public_url)

ollama_tunnel = ngrok.connect(11422, "http")
print("\nSetup complete. Ollama is hidden, but accessible locally on port 11422.")